In [1]:
import time
from IPython.display import clear_output
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from  lxml import etree
import json 
import os
from tqdm import tqdm
import traceback
import traceback
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

In [2]:
TIME=5

class crawling:
    def __init__(self,):
        self.home_page = "https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/1/ha-noi.html"
        self.root = "https://alonhadat.com.vn"
        self.page = 1

    def get_pages(self, url):
        bs = BeautifulSoup(self.driver.page_source)
        elements = bs.find_all("div", {"class": "ct_title"})
        if len(elements) == 0:
            self.avoid_captcha(url)
            bs = BeautifulSoup(self.driver.page_source)
            elements = bs.find_all("div", {"class": "ct_title"})
        data = []
        try:
            for feature in elements:
                feature = feature.find_all('a', href=True)
                data.append(feature[0]['href'])
        except Exception as e:
            print(e)
        return [self.root+link for link in data]
    
    def next_page(self):
        self.page += 1
        return self.home_page[:-5] + f"/trang--{self.page}.html"

    def get_id(self, url):
        bs = BeautifulSoup(self.driver.page_source)
        page = etree.HTML(str(bs))
        address = page.xpath("""//*[@id="left"]/div[1]/div[5]/div[2]/table/tbody/tr[1]""")
        num = 5
        while len(address) == 0 and num > 0:
            self.avoid_captcha(url)
            bs = BeautifulSoup(self.driver.page_source)
            page = etree.HTML(str(bs))
            address = page.xpath("""//*[@id="left"]/div[1]/div[5]/div[2]/table/tbody/tr[1]""")
            num-=1
        address = address[0]
        return address.getchildren()[1].text
    
    def gather(self, page_source):
        data = {}
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))

        # get price
        price = page.xpath("""//*[@id="left"]/div[1]/div[3]/span[1]/span[2]""")[0].text
        price = price.replace(",", ".")
        data['price'] = price

        # get area
        area = page.xpath("""//*[@id="left"]/div[1]/div[3]/span[2]/span[2]""")[0].text
        data['area'] = area
        
        # get address
        address = page.xpath("""//*[@id="left"]/div[1]/div[4]/span[2]""")[0].text
        address = address.replace(",", ".")
        data['address'] = address

        # get table information
        table = bs.find_all('div', {'class': 'infor'})[0]
        tr_instances = table.find_all('tr')
        for i in range(len(tr_instances)):
            td_instances = tr_instances[i].find_all('td')
            for j in range(3):
                data[str(td_instances[2*j].text).strip()] = str(td_instances[2*j+1].text)
        
        # get description
        description = bs.find_all('div', {'class': 'detail text-content'})[0].text
        data['description'] = description

        return data
    
    def avoid_captcha(self, page):
        self.driver.close()
        self.prepare_driver()
        self.driver.get(page)
        bs = BeautifulSoup(self.driver.page_source)
        iframe_name = bs.find_all("iframe", {"title": "reCAPTCHA"})[0]['name']
        self.driver.switch_to.frame(iframe_name)
        self.driver.find_element(By.XPATH, '''/html/body/div[2]/div[3]/div[1]/div/div/span''').click()
        print('...')
        time.sleep(20)
        self.driver.switch_to.default_content()
        self.driver.find_element(By.XPATH, '''/html/body/div[1]/form/button''').click()
        print('success')

    def prepare_driver(self):
        service = ChromeService(executable_path=ChromeDriverManager().install())
        options = webdriver.ChromeOptions() 
        options.add_argument('--disable-blink-features=AutomationControlled')
        self.driver = webdriver.Chrome(options=options, service=service)
        
    def run(self, start, num_of_pages):
        url = self.home_page
        self.page = 1
        if start != 1:
            self.page = start
            url = self.home_page[:-5] + f"/trang--{self.page}.html"
        if os.path.exists("data/index.json"):
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
        with open("log.txt", "w") as f:
            pass
        dataset = []
        for _ in range(num_of_pages):
            num = 10
            pages = []
            while num > 0:
                try:
                    self.driver.get(url)
                    # time.sleep(TIME)
                    pages = self.get_pages(url)
                    self.driver.delete_all_cookies()
                    break
                except Exception as e:
                    print(e)
                    num -= 1
            if num <= 0:
                url = self.next_page()
                continue
            for page in tqdm(pages):
                num = 1
                while num > 0:
                    try:
                        self.driver.get(page)
                        id = self.get_id(page)
                        if id in index: 
                            print(id, "exist")
                            break
                        data = self.gather(self.driver.page_source)
                        if data["Mã tin"] not in index:
                            dataset.append(data)
                            index.add(data["Mã tin"])
                        with open("log.txt", "a") as f:
                            f.write("Success:" + page +"\n")
                        self.driver.delete_all_cookies()
                        break
                    except Exception as e:
                        print(e, "unknown")
                        with open("log.txt", "a") as f:
                            f.write(f"Excution{5-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        with open("failed.txt", "a") as f:
                            f.write(f"Excution{5-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        self.driver.delete_all_cookies()
                        num -= 1
            url = self.next_page()
        if len(dataset) != 0:
            df = pd.DataFrame(dataset)
            df.to_csv(f"data/{start}_{start+num_of_pages}_{time.time()}.csv", index=False)
        with open("data/index.json", "w") as f:
            json.dump(list(index), f)
        self.driver.delete_all_cookies()
        
craw = crawling()

In [ ]:
# Cần bán: 11353 (the last page)
# Cho thuê: 1211 (the last page)
craw.prepare_driver()
for i in range(1, 11354):
    craw.run(i, 1)
    clear_output()